<img align="left" src="https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcS7ovkoWSZdub9adLaiuFEdXYUzjZ6bSSNXPA6vT3z3gmCccuMr"></img>




# Corona Virus

   * Coronaviruses are **zoonotic** viruses (means transmitted between animals and people).  
   * Symptoms include from fever, cough, respiratory symptoms, and breathing difficulties. 
   * In severe cases, it can cause pneumonia, severe acute respiratory syndrome (SARS), kidney failure and even death.
   * Coronaviruses are also **asymptomatic**, means a person can be a carrier for the infection but experiences no symptoms

# Novel coronavirus (nCoV)
* A **novel coronavirus (nCoV)** is a new strain that has not been previously identified in humans.

# COVID-19 (Corona Virus Disease 2019)
* Caused by a **SARS-COV-2** corona virus.  
* First identified in **Wuhan, Hubei, China**. Earliest reported symptoms reported in **November 2019**. 
* First cases were linked to contact with the Huanan Seafood Wholesale Market, which sold live animals. 
* On **30 January** the WHO declared the outbreak to be a **Public Health Emergency of International Concern** 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd
%matplotlib inline
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_dark"
from plotly.subplots import make_subplots


import folium 
from folium import plugins
from tqdm.notebook import tqdm as tqdm

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
cleaned_data=pd.read_csv('/kaggle/input/corona-virus-report/covid_19_clean_complete.csv')
cleaned_data.head(15)

**Preprocessing**

In [ ]:
# cases 
cases = ['Confirmed', 'Deaths', 'Recovered', 'Active']

# Active Case = confirmed - deaths - recovered
cleaned_data['Active'] = cleaned_data['Confirmed'] - cleaned_data['Deaths'] - cleaned_data['Recovered']

# filling missing values 
cleaned_data[['Province/State']] = cleaned_data[['Province/State']].fillna('')
cleaned_data[cases] = cleaned_data[cases].fillna(0)

cleaned_data.head()


# World Stats

In [ ]:
Date_df=cleaned_data.groupby('Date')['Confirmed','Recovered', 'Deaths', 'Active'].sum().reset_index()
Date_df=Date_df.sort_values(by=['Confirmed'])
Date_df.head(40)

**Total Cases**

In [ ]:
temp = Date_df.melt(id_vars="Date", value_vars=['Recovered', 'Deaths', 'Active'],
                 var_name='case', value_name='count')
fig = px.area(temp, x="Date", y="count", color='case',
             title='Cases over time: Area Plot', color_discrete_sequence = ['cyan', 'red', 'orange'])
fig.update_xaxes(tick0=4, dtick=4)

fig.show()

As per the graph we could analysis that Active cases were on a constant growth between 11th februrary,2020 and 12th march,2020 but eventually started growing at an increasing rate since 14th march,2020 till date as per data.

**Daily new cases**

In [ ]:
for column in Date_df[['Confirmed']]:
   # Select column contents by column name using [] operator
   columnSeriesObj = Date_df[column]
#    print('Colunm Name : ', column)
#    print('Column Contents : ', columnSeriesObj.values)
   new_cases=columnSeriesObj.values
my_list=[]
for i in range(0,len(new_cases)-1):
    new_value=new_cases[i+1]-new_cases[i]
    my_list.append(new_value)
my_list.insert(0,0)

print(len(my_list))
print(Date_df.shape)
Date_df['New_cases'] = np.array(my_list)


In [ ]:
import plotly.express as px
fig = go.Figure()
fig.add_trace(go.Bar(
    x=Date_df['Date'],
    y=Date_df['New_cases'],
    name='New_cases',
    marker_color='grey'
))
fig.show()

Graph shows the current status of total number of cases been increased or decreased on daily basis. 
as per it we could see the first rise was on 13th feburary,2020 upto 13,639 cases , whereas second rise could be seen exact after a month on 13th march,2020 upto 16,853. Highest peek was on 4th april,2020 upto 1,01,488.Since 13th march storm didn't ended it began to rise as per the date in worldwide.

**Newly Infected vs. Newly Recovered**

In [ ]:
for column in Date_df[['Recovered']]:
   # Select column contents by column name using [] operator
   columnSeriesObj = Date_df[column]
#    print('Colunm Name : ', column)
#    print('Column Contents : ', columnSeriesObj.values)
   new_recover=columnSeriesObj.values
my_list1=[]
for i in range(0,len(new_recover)-1):
    new_value=new_recover[i+1]-new_recover[i]
    my_list1.append(new_value)
my_list1.insert(0,0)
print(len(my_list1))
print(Date_df.shape)
Date_df['New_recover'] = np.array(my_list1)
Date_df.head(5)

data in variables like Recovered , Active , Deaths and Confirmed were the cumulative one adding old one too . in order to get the bew cases count in confirmed and recovered new variabled were formed to avoid duplicacy.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=Date_df['Date'], y=Date_df['New_cases'],
                    mode='lines+markers',
                    name='New_cases'))
fig.add_trace(go.Scatter(x=Date_df['Date'], y=Date_df['New_recover'],
                    mode='lines+markers',
                    name='New_recover'))
fig.update_xaxes(tick0=4, dtick=4)
fig.show()

In this graph less of variation can be seen in case of newly recovered cases and high rate of variation in new cases as new one were being added to the list on daily basis worldwide.
first rise of new cases was between 12th and 15th feburary,2020 and since after 8th of march it kept rising at a normal rate.

**Outcome of Cases (Recovery or Death)**

In [ ]:
Date_df['Closed_cases']=Date_df['Recovered']+Date_df['Deaths']

In [ ]:
Date_df['Recover_percent']=(Date_df['Recovered']/Date_df['Closed_cases'])*100
Date_df['Death_percent']=(Date_df['Deaths']/Date_df['Closed_cases'])*100
Date_df


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=Date_df['Date'], y=Date_df['Recover_percent'],
                    mode='lines+markers',
                    name='Recover %',line=dict(color='green')))
fig.add_trace(go.Scatter(x=Date_df['Date'], y=Date_df['Death_percent'],
                    mode='lines+markers',
                    name='Death %',line=dict(color='red')))
fig.update_xaxes(tick0=4, dtick=4)
fig.update_layout(title='Output of Closed Cases(Recovery OR Death)',
                   xaxis_title='Date',
                   yaxis_title='Percentage')
fig.show()

Percentage is a better way to represent the recoverd cases and deaths during COVID-19 as it gives a bet=ter visualization of the situation.

there was time betwwen 22nd january and 31st january,2020 when percentage of people recovered and persons who dierd were 50%. Fortunately the percebtage of recovered people rised as on 6th of march,2020 people who recovered were 94.167% whereas the people who died were just 5.832%.
Lately it reduced but at a decreasing rate as percentage of people recoverd was 78.34% as to 21.65% of people who died as on 7th april,2020.


**Countries Cases Distribution**

In [ ]:
new_df=cleaned_data.loc[cleaned_data['Country/Region'] == 'China'] 
# v=cleaned_data.groupby('Country/Region')['Active'].sum().reset_index()
# v = v.melt(id_vars="Country/Region", value_vars=['Active'],
#                  var_name='case', value_name='count')
new_df1=cleaned_data.loc[cleaned_data['Date'] == '4/7/20'] 
res_df=new_df1.groupby('Country/Region')['Confirmed','Recovered', 'Deaths', 'Active'].sum().reset_index()
fig = px.pie(res_df, values='Confirmed', names='Country/Region', title='Countries Cases Distribution')
fig.show()

maximum of people found infected with COVID-19 was in US followed by the countries like Spain , Italy , France and Germany.
why US is on top of the list despite the fact disease was born in china 
1- US lacks enough testing capacity
2- emerged as 'hotspots' for exhibiting growth
3-US social distancing ,easure remain less strict that of those other countries.

**TOP 10 Countries with most confirmed cases**

In [ ]:
top_10=res_df.sort_values(by=['Confirmed'],ascending=False)[0:10]

In [ ]:
import plotly.express as px
fig = px.bar(top_10, x='Country/Region', y='Confirmed')
fig.show()


Going with the maximum contribution in this panedemic disease order remains the same for maximum number of confirmed cases too.

Top 10 countries with most death cases

In [ ]:
top_10_death=res_df.sort_values(by=['Deaths'],ascending=False)[0:10]
top_10_death.head(5)
import plotly.express as px
fig = px.bar(top_10_death, x='Country/Region', y='Deaths')
fig.show()

Demographics, social behavior and lower testing capacity are just some of the reasons why Italy and Spain have the highest number of deaths in the world from the coronavirus, health experts told CNBC. 
reasons maybe like 
1- Slow response - there were a lot of spread before people realized
2- testing capacity - as how many people are being tested for virus.In places where many people are being tested, and quickly, such as China, the number of deaths is not as high as those seen in Italy and Spain, where only citizens showing symptoms of the coronavirus are being tested rest didn't bothered to.
3- population (elderly) - There was a “double combination of risk factors” in Italy .The first group of people to get hit by the virus in the country were the elderly.

OECD data shows that Italy has the second oldest population in the world, after Japan. Those aged above 60 are believed to be at a higher risk of having severe symptoms from the virus.

**Case Progression of top 5 countries**

In [ ]:
us_data=cleaned_data.loc[cleaned_data['Country/Region']=='US']
us_data=us_data.groupby('Date')['Confirmed','Recovered', 'Deaths', 'Active'].sum().reset_index()
us_data=us_data.sort_values(by=['Confirmed'])


Spain_data=cleaned_data.loc[cleaned_data['Country/Region']=='Spain']
Spain_data=Spain_data.groupby('Date')['Confirmed','Recovered', 'Deaths', 'Active'].sum().reset_index()
Spain_data=Spain_data.sort_values(by=['Confirmed'])



Italy_data=cleaned_data.loc[cleaned_data['Country/Region']=='Italy']
Italy_data=Italy_data.groupby('Date')['Confirmed','Recovered', 'Deaths', 'Active'].sum().reset_index()
Italy_data=Italy_data.sort_values(by=['Confirmed'])



France_data=cleaned_data.loc[cleaned_data['Country/Region']=='France']
France_data=France_data.groupby('Date')['Confirmed','Recovered', 'Deaths', 'Active'].sum().reset_index()
France_data=France_data.sort_values(by=['Confirmed'])



Germany_data=cleaned_data.loc[cleaned_data['Country/Region']=='Germany']
Germany_data=Germany_data.groupby('Date')['Confirmed','Recovered', 'Deaths', 'Active'].sum().reset_index()
Germany_data=Germany_data.sort_values(by=['Confirmed'])


fig = go.Figure()
fig.add_trace(go.Scatter(x=us_data['Date'][30:], y=us_data['Confirmed'][30:],
                    mode='markers',
                    name='US'))
fig.add_trace(go.Scatter(x=us_data['Date'][30:], y=Spain_data['Confirmed'][30:],
                    mode='markers',
                    name='Spain'))
fig.add_trace(go.Scatter(x=us_data['Date'][30:], y=Italy_data['Confirmed'][30:],
                    mode='markers',
                    name='Italy'))
fig.add_trace(go.Scatter(x=us_data['Date'][30:], y=France_data['Confirmed'][30:],
                    mode='markers',
                    name='France'))
fig.add_trace(go.Scatter(x=us_data['Date'][30:], y=Germany_data['Confirmed'][30:],
                    mode='markers',
                    name='Germany'))
fig.update_xaxes(tick0=4, dtick=4)
fig.show()

cases progression for confirmed cases in our top 5 countries
us ,spain , italy , france and germany
top of the list is owned by US for having maximum number of confirmed cases , followed by Spain and Italy being on almost same number of cases. and lst of the top 5 list is france.

**Death progression of top countries**

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=us_data['Date'][30:], y=us_data['Deaths'][30:],
                    mode='markers',
                    name='US'))
fig.add_trace(go.Scatter(x=us_data['Date'][30:], y=Spain_data['Deaths'][30:],
                    mode='markers',
                    name='Spain'))
fig.add_trace(go.Scatter(x=us_data['Date'][30:], y=Italy_data['Deaths'][30:],
                    mode='markers',
                    name='Italy'))
fig.add_trace(go.Scatter(x=us_data['Date'][30:], y=France_data['Deaths'][30:],
                    mode='markers',
                    name='France'))
fig.add_trace(go.Scatter(x=us_data['Date'][30:], y=Germany_data['Deaths'][30:],
                    mode='markers',
                    name='Germany'))
fig.update_xaxes(tick0=4, dtick=4)
fig.show()

death pole is topped by Italy due to same reasons mentioned above - slow response , testing capacity and more of elderly population in the country.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=us_data['Date'][30:], y=us_data['Active'][30:],
                    mode='markers',
                    name='US'))
fig.add_trace(go.Scatter(x=us_data['Date'][30:], y=Spain_data['Active'][30:],
                    mode='markers',
                    name='Spain'))
fig.add_trace(go.Scatter(x=us_data['Date'][30:], y=Italy_data['Active'][30:],
                    mode='markers',
                    name='Italy'))
fig.add_trace(go.Scatter(x=us_data['Date'][30:], y=France_data['Active'][30:],
                    mode='markers',
                    name='France'))
fig.add_trace(go.Scatter(x=us_data['Date'][30:], y=Germany_data['Active'][30:],
                    mode='markers',
                    name='Germany'))
fig.update_xaxes(tick0=4, dtick=4)
fig.show()

since 24th march maximum of active cases are in US due to US social distancing measure remian less strict that those of other countries.
And it emerged as on of the ehibiting hotspots for the corona's growth rate there.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=us_data['Date'][30:], y=us_data['Recovered'][30:],
                    mode='markers',
                    name='US'))
fig.add_trace(go.Scatter(x=us_data['Date'][30:], y=Spain_data['Recovered'][30:],
                    mode='markers',
                    name='Spain'))
fig.add_trace(go.Scatter(x=us_data['Date'][30:], y=Italy_data['Recovered'][30:],
                    mode='markers',
                    name='Italy'))
fig.add_trace(go.Scatter(x=us_data['Date'][30:], y=France_data['Recovered'][30:],
                    mode='markers',
                    name='France'))
fig.add_trace(go.Scatter(x=us_data['Date'][30:], y=Germany_data['Recovered'][30:],
                    mode='markers',
                    name='Germany'))
fig.update_xaxes(tick0=4, dtick=4)
fig.show()

spain is on the top of the list to get recovered as extention of state of alarm was announced that could keep citizens at home until at least 26th april.
Spanish Prime Minister Pedro Sánchez announced on Saturday the total lockdown of all 47 million Spaniards apart from those who are working in essential sectors, in a bid to halt the spread of the coronavirus and to reduce the congestion of the country’s intensive care units (ICUs). 

# **Modeling**

#  Train/Val split 

> First, we need to create miniature training and validation sets to train and validate our models. I will take the last 15 days as the validation data and the 55 days before that as the training data. We need to predict the Cases in the validation data using the Cases in the training data.**

In [ ]:
train_dataset = pd.read_csv('../input/time-covid19/time_series_covid_19_confirmed.csv')
drop_clo = ['Province/State','Country/Region','Lat','Long']
train_dataset=train_dataset.drop(drop_clo,axis=1)
datewise= list(train_dataset.columns)
val_dataset = train_dataset[datewise[-15:]]

# Naive approach
The first approach is the very simple naive approach. It simply forecasts the next day's cases as the current day's cases. The model can be summarized as:

<img src="https://i.imgur.com/r8wjrzk.png" width="120px">

In the above equation, yt+1 is the predicted value for the next day's cases and yt is today's cases. The model predicts tomorrow's cases as today's sales. Now let us see how this simple model performs on our miniature dataset. The training data is in blue, validation data in orange, and predictions in green.

In [ ]:
date_array=np.asarray(Date_df['Date'])
fig = make_subplots(rows=3, cols=1)

fig.add_trace(
    go.Scatter(x=date_array[:-15], mode='lines+markers', y=train_dataset.loc[0].values[:-15], marker=dict(color="dodgerblue"), showlegend=False,),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=date_array[-15:], y=val_dataset.loc[0].values, mode='lines+markers', marker=dict(color="darkorange"), showlegend=False,),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=date_array[:-15], mode='lines+markers', y=train_dataset.loc[1].values[:-15], marker=dict(color="dodgerblue"), showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=date_array[-15:], y=val_dataset.loc[1].values, mode='lines+markers', marker=dict(color="darkorange"), showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=date_array[:-15], mode='lines+markers', y=train_dataset.loc[2].values[:-15], marker=dict(color="dodgerblue"), showlegend=False),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=date_array[-15:], y=val_dataset.loc[2].values, mode='lines+markers', marker=dict(color="darkorange"), showlegend=False),
    row=3, col=1
)

fig.update_layout(height=1200, width=800, title_text="Train (blue) vs. Validation (orange) sales")
fig.show()

In [ ]:
predictions = []
for i in range(len(val_dataset.columns)):
    if i == 0:
        predictions.append(train_dataset[train_dataset.columns[-16]].values)
    else:
        predictions.append(val_dataset[val_dataset.columns[i-1]].values)
    
predictions = np.transpose(np.array([row.tolist() for row in predictions]))
error_naive = np.linalg.norm(predictions[:] - val_dataset.values[:])/len(predictions[:])

In [ ]:
pred_1 = predictions[0]
pred_2 = predictions[1]
pred_3 = predictions[2]

fig = make_subplots(rows=3, cols=1)

fig.add_trace(
    go.Scatter(x=date_array[:-15], mode='lines+markers', y=train_dataset.loc[0].values[:-15], marker=dict(color="dodgerblue"),name="Train"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=date_array[-15:], y=val_dataset.loc[0].values, mode='lines+markers', marker=dict(color="darkorange"), name="Validation"),
    row=1, col=1,
)

fig.add_trace(
    go.Scatter(x=date_array[-15:], y=pred_1, mode='lines', marker=dict(color="seagreen"),
               name="Pred"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=date_array[:-15], mode='lines+markers', y=train_dataset.loc[1].values[:-15], marker=dict(color="dodgerblue"), showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=date_array[-15:], y=val_dataset.loc[1].values, mode='lines+markers', marker=dict(color="darkorange"), showlegend=False),
    row=2, col=1
)


fig.add_trace(
    go.Scatter(x=date_array[-15:], y=pred_2, mode='lines', marker=dict(color="seagreen"), showlegend=False,
               name="Denoised signal"),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=date_array[:-15], mode='lines+markers', y=train_dataset.loc[2].values[:-15], marker=dict(color="dodgerblue"), showlegend=False),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=date_array[-15:], y=val_dataset.loc[2].values, mode='lines+markers', marker=dict(color="darkorange"), showlegend=False),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=date_array[-15:], y=pred_3, mode='lines', marker=dict(color="seagreen"), showlegend=False,
               name="Denoised signal"),
    row=3, col=1
)

fig.update_layout(height=1200, width=800, title_text="Naive approach")
fig.show()

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
rms = sqrt(mean_squared_error(predictions[:3] ,val_dataset.values[:3]))
print(rms)

# Holt's Linear model

### Forecast, level, and trend equations respectively
<img src="https://i.imgur.com/MHgcgGo.png" width="180px">
<img src="https://i.imgur.com/3ImRHEO.png" width="300px">
<img src="https://i.imgur.com/XExnvMX.png" width="300px">


In [ ]:
model_train=Date_df.iloc[:int(Date_df.shape[0]*0.90)]
valid=Date_df.iloc[int(Date_df.shape[0]*0.90):]

In [ ]:
import statsmodels.api as sm
from statsmodels.tsa.api import Holt,SimpleExpSmoothing,ExponentialSmoothing
holt=Holt(np.asarray(model_train["Active"])).fit(smoothing_level=0.2, smoothing_slope=0.8)
y_pred=valid.copy()

In [ ]:
valid.tail(5)


In [ ]:
import matplotlib.pyplot as plt
y_pred["Holt"]=holt.forecast(len(valid))
#model_scores.append(np.sqrt(mean_squared_error(y_pred["Confirmed"],y_pred["Holt"])))
print("Root Mean Square Error Holt's Linear Model: ",np.sqrt(mean_squared_error(y_pred["Confirmed"],y_pred["Holt"])))

In [ ]:
ls=list(valid['Date'])

In [ ]:
ls.extend(['4/10/20','4/11/20','4/12/20','4/13/20','4/14/20','4/15/20','4/16/20','4/17/20','4/18/20'])

In [ ]:
x=holt.forecast(len(valid)+len(ls))
x

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=model_train['Date'], y=model_train['Active'],
                    mode='lines+markers',
                    name='Train ',line=dict(color='green')))
fig.add_trace(go.Scatter(x=ls, y=valid['Active'],
                    mode='lines+markers',
                    name='validation ',line=dict(color='red')))
fig.add_trace(go.Scatter(x=ls, y=x,
                    mode='lines+markers',
                    name='predicted ',line=dict(color='white')))
fig.update_xaxes(tick0=4, dtick=4)
fig.update_layout(title='Active cases',
                   xaxis_title='Date',
                   yaxis_title='Count')
fig.show()

# Moving on to Top 5 Countries

**1. US**

In [ ]:
us_data=cleaned_data.loc[cleaned_data['Country/Region']=='US']
us_data=us_data.groupby('Date')['Confirmed','Recovered', 'Deaths', 'Active'].sum().reset_index()
us_data=us_data.sort_values(by=['Confirmed'])


In [ ]:
state_data=cleaned_data.loc[cleaned_data['Country/Region']=='US']
state_data.head(5)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=us_data['Date'], y=us_data['Active'],
                    mode='lines+markers',
                    name='Active'))
fig.add_trace(go.Scatter(x=us_data['Date'], y=us_data['Deaths'],
                    mode='lines+markers',
                    name='Deaths'))
fig.update_xaxes(tick0=4, dtick=4)
fig.show()

In [ ]:
new_df=cleaned_data.loc[cleaned_data['Country/Region'] == 'China'] 
# v=cleaned_data.groupby('Country/Region')['Active'].sum().reset_index()
# v = v.melt(id_vars="Country/Region", value_vars=['Active'],
#                  var_name='case', value_name='count')
new_df1=cleaned_data.loc[cleaned_data['Date'] == '4/7/20'] 
res_df=new_df1.groupby('Country/Region')['Confirmed','Recovered', 'Deaths', 'Active'].sum().reset_index()

In [ ]:
new_df1=cleaned_data.loc[cleaned_data['Date'] == '4/7/20'] 

In [ ]:
res_df=new_df1.groupby('Country/Region')['Confirmed','Recovered', 'Deaths', 'Active'].sum().reset_index()

In [ ]:

import plotly.graph_objects as go



fig = go.Figure()
fig.add_trace(go.Bar(
    x=top_10['Country/Region'],
    y=top_10['Confirmed'],
    name='Confirmed',
    marker_color='indianred'
))
fig.add_trace(go.Bar(
    x=top_10['Country/Region'],
    y=top_10['Active'],
    name='Active',
    marker_color='lightsalmon'
))

fig.add_trace(go.Bar(
    x=top_10['Country/Region'],
    y=top_10['Deaths'],
    name='Deaths',
    marker_color='red'
))
fig.add_trace(go.Bar(
    x=top_10['Country/Region'],
    y=top_10['Recovered'],
    name='Recovered',
    marker_color='green'
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.show()

In [ ]:
import plotly.express as px
fig = px.bar(top_10, x="sex", y="total_bill", color="smoker", barmode="group",
             facet_row="time", facet_col="day",
             category_orders={"day": ["Thur", "Fri", "Sat", "Sun"],
                              "time": ["Lunch", "Dinner"]})
fig.show()